In [8]:
import numpy as np 
import pandas as pd
from helpers import * 
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)
ufc = pd.read_csv('ufc-master.csv')

In [9]:
cols_to_norm = ['B_age', 'R_ev', 'R_odds', 'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'perf_diff']

In [10]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()

In [11]:
feats, targ = fine_tuning_setup(ufc, one_or_two=True)

In [13]:
feats_std = feats.copy()

In [14]:
scale.fit(feats_std[cols_to_norm])

StandardScaler()

In [15]:
feats_std[cols_to_norm] = scale.transform(feats_std[cols_to_norm])

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0, n_estimators=175, max_features='log2', criterion='gini')


In [17]:
ml = {'rf' : rf}

In [18]:
simulate(feats, targ, ml)

rf
0.7386706948640483


In [19]:
simulate(feats_std, targ, ml)

rf
0.7401812688821753


In [20]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 175, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy'],
    # Add any other parameters you want to tune
}

In [22]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')

In [23]:
grid_search.fit(feats_std, targ)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_features='log2',
                                              n_estimators=175,
                                              random_state=0),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 175, 200]},
             scoring='accuracy')

In [26]:
grid_search.best_score_

0.6700906344410876

In [27]:
grid_search.best_params_

{'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 175}

In [28]:
rf2 = RandomForestClassifier(n_estimators=175, criterion='gini', max_features='auto')

In [29]:
simulate(feats_std, targ, ml)

rf
0.7401812688821753


In [30]:
ml2 = {'rf' : rf2}

In [32]:
feats_std['better_rank'] = ufc['better_rank']

In [35]:
better_dict = {
	'neither' : 0,
	'Red' : -1,
	'Blue' : 1
}

In [37]:
feats_std['better_rank'] = [better_dict[val] for val in feats_std['better_rank']]

In [38]:
simulate(feats_std, targ, ml)

rf
0.7265861027190332


In [40]:
feats_std.head()

,B_current_win_streak,R_win_by_Submission,B_win_by_Decision_Unanimous,R_win_by_Decision_Unanimous,R_current_lose_streak,B_win_by_TKO_Doctor_Stoppage,win_dif,B_win_by_Decision_Split,B_wins,R_Stance,...,R_Weight_lbs,R_current_win_streak,R_age,empty_arena,R_win_by_Decision_Split,R_draw,lose_streak_dif,B_draw,perf_diff,better_rank
0,0,3,0,3,2,0,-5,0,6,1,...,0.104498,0,0.606910,1,1,0,0,0,-0.187101,-1
1,0,5,6,0,1,0,3,2,9,1,...,1.103020,0,0.850410,1,0,0,0,0,1.783151,0
2,2,2,4,2,0,0,1,0,5,1,...,-0.323440,4,-0.610588,1,0,0,0,0,0.443906,0
3,2,1,0,1,1,0,-2,0,2,2,...,-1.179316,0,1.337409,1,0,0,-1,0,-0.972829,0
4,3,0,2,2,1,0,-2,1,3,1,...,-0.894024,0,1.824409,1,2,1,-1,0,-0.389060,0


In [42]:
feats_std = feats_std.drop(['better_rank'], axis=1)

In [43]:
feats_std['draw_dif'] = feats_std['R_draw'] - feats_std['B_draw']

In [45]:
feats_std = feats_std.drop(['R_draw', 'B_draw'], axis=1)

In [47]:
simulate(feats_std, targ, ml)

rf
0.7371601208459214
